# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Embedded-in-Jupyter-notebook" data-toc-modified-id="Embedded-in-Jupyter-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Embedded in Jupyter notebook</a></div><div class="lev1 toc-item"><a href="#Using-GTK" data-toc-modified-id="Using-GTK-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Using GTK</a></div>

# Embedded in Jupyter notebook

In [ ]:
%matplotlib notebook

import matplotlib as mpl
import matplotlib.pyplot as plt

import time
import threading

import ipywidgets as ipw
import numpy as np
import pandas as pd


fig, axis = plt.subplots()

axis._get_lines()
stop_event = threading.Event()
np.random.seed(0)
data = []


def _draw():
    while True:
        if data:
            pd.Series(np.concatenate(data)).plot(ax=axis)
            fig.canvas.show()
        data.append(np.random.rand(10))
        if stop_event.wait(.5):
            break
    stop_event.clear()
    start.disabled = False
    stop.disabled = True
        
    
def _start(*args):
    start.disabled = True
    stop.disabled = False
    thread = threading.Thread(target=_draw)
    thread.daemon = True
    thread.start()

start = ipw.Button(description='Start')
start.on_click(_start)
stop = ipw.Button(description='Stop')
stop.on_click(lambda *args: stop_event.set())
clear = ipw.Button(description='Clear')
def _clear(*args):
    axis.cla()
    for i in xrange(len(data)):
        data.pop()
clear.on_click(_clear)

widget = ipw.HBox([start, stop, clear])
widget

------------------------------------------------------------------------

# Using GTK

In [14]:
# See [here][1] for info on handling mouse events in the
# GTK matplotlib interactive widget.
#
# See [here][2] for general usage info for the GTK matplotlib
# interactive widget.
#
# **Note**: Use right-click and drag while in "pan" mode to zoom.
#
# [1]: https://matplotlib.org/users/event_handling.html
# [2]: https://matplotlib.org/users/navigation_toolbar.html
import gobject
import gtk

from matplotlib.figure import Figure
from matplotlib.backends.backend_gtkagg import FigureCanvasGTKAgg as FigureCanvas
from matplotlib.backends.backend_gtkagg import NavigationToolbar2GTKAgg as NavigationToolbar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import threading
import datetime as dt
from pygtkhelpers.delegates import SlaveView

import matplotlib as mpl
import matplotlib.ticker
import si_prefix as si

A_formatter = mpl.ticker.FuncFormatter(lambda x, *args: '%sA' %
                                       si.si_format(x, 2))
s_formatter = mpl.ticker.FuncFormatter(lambda x, *args: '%ss' %
                                       si.si_format(x))


class StreamingPlot(SlaveView):
    def __init__(self, data=None):
        super(StreamingPlot, self).__init__()
        if data is not None:
            self.data = data
        else:
            self.data = []
        
    def create_ui(self):
        def _destroy(*args):
            stop_event.set()
            
        self.widget.connect('destroy', _destroy)
#         self.widget.set_default_size(640, 480)

        vbox = gtk.VBox()
        self.widget.add(vbox)

        with mpl.style.context('fivethirtyeight'):
            fig, axis = plt.subplots()

            canvas = FigureCanvas(fig)  # a gtk.DrawingArea
            vbox.pack_start(canvas)
            toolbar = NavigationToolbar(canvas, win)
            vbox.pack_start(toolbar, False, False)
            stop_event = threading.Event()
            np.random.seed(0)
            axis.xaxis.set_major_formatter(s_formatter)
            axis.yaxis.set_major_formatter(A_formatter)

            def _draw():
                delta_t = dt.timedelta(seconds=.1)
                samples_per_plot = 5

                line = None
                while True:
                    if self.data:
                        plot_data = pd.concat(self.data)
                        absolute_times = plot_data.index.to_series()
                        relative_times = ((absolute_times - absolute_times.iloc[0])
                                          .dt.total_seconds())
                        plot_data.index = relative_times
                        if line is None:
                            line = axis.plot(plot_data.index.values,
                                             plot_data.values)[0]
                        else:
                            line.set_data(plot_data.index.values, plot_data.values)

                        def _draw_i(axis, plot_data):
                            axis.relim()
                            axis.set_xlim(right=plot_data.index[-1])
                            axis.set_ylim(auto=True)
                            axis.autoscale_view(True, True, True)
                            axis.get_figure().canvas.draw()
                        gobject.idle_add(_draw_i, axis, plot_data)

                    time_0 = dt.datetime.now()
                    values_i = np.random.rand(samples_per_plot)
                    absolute_times_i = pd.Series([time_0 + i * delta_t
                                                  for i in xrange(len(values_i))])
                    data_i = pd.Series(values_i, index=absolute_times_i)
                    self.data.append(data_i)
                    if stop_event.wait(samples_per_plot * delta_t.total_seconds()):
                        break
                stop_event.clear()

                def _button_states():
                    start.set_label('Continue')
                    start.props.sensitive = True
                    clear.props.sensitive = True
                    stop.props.sensitive = False
                gobject.idle_add(_button_states)

            def _start(*args):
                def _button_states():
                    start.props.sensitive = False
                    clear.props.sensitive = False
                    stop.props.sensitive = True
                gobject.idle_add(_button_states)

                thread = threading.Thread(target=_draw)
                thread.daemon = True
                thread.start()

            start = gtk.Button('Start')
            start.connect("clicked", _start)

            stop = gtk.Button('Pause')
            stop.connect("clicked", lambda *args: stop_event.set())
            stop.props.sensitive = False

            clear = gtk.Button('Reset')
            def _clear(*args):
                for line_i in axis.get_lines():
                    line_i.remove()
                for i in xrange(len(self.data)):
                    self.data.pop()
                def _reset_ui(*args):
                    start.set_label('Start')
                    clear.props.sensitive = False
                    fig.canvas.draw()
                gobject.idle_add(_reset_ui)
            clear.connect("clicked", _clear)
            clear.props.sensitive = False

            button_box = gtk.HBox()
            for widget_i in (start, stop, clear):
                button_box.pack_start(widget_i, False, False)
            vbox.pack_start(button_box, False, False)

            axis.set_ylabel('Current')
            axis.set_xlabel('Time')

        self.fig = fig
    
    def on_resize(self):
        gobject.idle_add(self.fig.tight_layout)
        
win = gtk.Window()
win.set_default_size(800, 600)
view = StreamingPlot()
win.add(view.widget)
win.connect('check-resize', lambda *args: view.on_resize())
win.set_position(gtk.WIN_POS_MOUSE)
win.show_all()
view.fig.tight_layout()
win.connect('destroy', gtk.main_quit)

gtk.gdk.threads_init()
gtk.gdk.threads_enter()
gtk.main()
gtk.gdk.threads_leave()